# Cryptography

### Team
<ol>
    <li> Mark Pearson </li>
    <li> Ronan Geraghty</li>
    <li> Milan Patel </li>
    <li> Will Steynor </li>
</ol>



In [1]:
from NumberFunctions import *
import numpy as np
import random as ran

### Core 2
RSA Sign Message

We know totient(prime) = prime - 1, and also totient is a multiplicative funciton.

So totient(prime1 x prime2) = (prime1 - 1)(prime2 - 1)

n-bit prime means it takes n bits of storage to store

### Modular Inverse
Solve, 
> $ax = 1 \pmod{m} $ for  $x \in \{1,2,3,...,m-1\}$


Proof

We have, where $x,y \in \mathbb{Z}$ are calculated by the Euclidean algorithm.
> $ax+by = gcd(a,b)$

We only need to take into account when $gcd(a,b)=1$ as $e$ is chosen so $gcd(e,\phi{N})=1$

Let $m=b$, where $gcd(a,m) = 1$

So we get
> \begin{align*}
ax+my &= 1\\
ax + my &\equiv 1\pmod{m}\\
ax &\equiv 1\pmod{m}
\end{align*}

Therefore, $x$ is a modular inverse of $a$ for $m$

However, by using the Euclidean algorithum sometimes $x$ is not in the set $\{1,2,...,m-1\}$. This is a problem as if $x$ is negative when we use it later for the RSA signiture we need to take the modulo of $int^{x}$, to do this we must have $int = int^{x}$

Let us consider the case where $x \notin \{1,2,...,m-1\}$, Let $d \in \mathbb{Z}$
> \begin{align*}
ax &\equiv 1\pmod{m}\\
ax + adm &\equiv 1\pmod{m}\\
a(x + dm) &\equiv 1\pmod{m}
\end{align*}

Set $x\prime = x + dm \iff x\prime = x \pmod{m}$

We know $x\prime \in \{1,2,...,m-1\}$


In [15]:
def totient(n):
    '''return the totient of n,
        number of postive integers < n which are coprime to n.'''
    num = 0
    for i in range(1,n):
        if gcd(i,n) == 1:
            num += 1
    return num

#Use hash
def my_hash(message):
    '''Turn message to bits
    Then turn into int h'''
    return abs(hash(message))

def modular_inverse(a,n):
    '''Solves ax = 1 mod(n) for x in {1,2,3,...,m-1}'''
    gcd, x, y = gcd_ext(a, n)
    if gcd == 1:
        return x%n
    else:
        return "gcd not = 1"


def primality_test(N, t):
    for i in range(0,t):
        a = ran.randrange(0,N-1)
        if a**(N-1) %N != 1:
            return "composite"
    return "prime"





In [16]:
#Tests

assert totient(7) == 6
assert totient(35) == 24
assert totient(5321) == 4992



assert modular_inverse(15,7) == 1

a=3487
n=9837
b=modular_inverse(a,n)
assert (a*b)%n == 1

#This test gets negative value originally, but inverse needs to be inbetween {1,2,..,m-1}
a = 3231
n = 12334
assert modular_inverse(a,n) == 11895

Need to gen primes of n bit lenght, idea pick number in range check if it is prime. However, with big primes doesn't work very fast. Use proboballistic approach therefore find prime which has a high prob of being prime(longer it works the higher prob of being prime).

Using Miller Rabin explain, proof

In [17]:
def random_prime(n_bit, t, trails = 500):
    '''Returns a uniform n-bit prime, with probability at most 2^(-t) getting a composite'''
    for i in range(0, trails):
        p = np.random.randint(0,2,n_bit-2)
        b = "".join(str(i) for i in p)
        my_int = int("1" + b + "1", 2) #Want at least n bit and gives odd        
        if is_prime_miller_rabin(my_int, t):
            return my_int
    return "Fail"

def is_prime_miller_rabin(N,t):
    '''If N is prime, test always outputs True(prime)
    If N is composite outputs False(not prime) except with probability at most 2^(-t)'''
    
    if N == 2:
        return True # Prime
    
    if N % 2 == 0:
        return False # Comp
    
    d, s = calc_d_s(N)
    for j in range(0,t):
        a = ran.randrange(2,N-1)
        if is_composite(N, d, s, a):
            return False #Comp
    return True #Prime
    
        
def is_composite(N, d, s, a):
    '''Evaulate a^d != +-1 mod(N), and, a^(d*2^r) != -1 mod(N) for r in {1,2,...,s-1},
    However, if output False does not mean prime could be strong liar'''
    remain = pow(a, d, N)
    if (remain != 1) and (remain != N - 1):
        for r in range(1,s):
            remain = pow(remain, 2, N)
            if remain != N-1:
                pass
            else:
                return False    
    else:
        return False
    
    return True
    
def calc_d_s(N):
    s = 0
    Num = N - 1
    while Num%2 == 0:
        Num //= 2
        s += 1
    d = Num
    return int(d), int(s)

In [18]:
#Tests
assert calc_d_s(71)[0] == 35
assert calc_d_s(71)[1] == 1
assert calc_d_s(127)[0] == 63
assert calc_d_s(127)[1] == 1
assert calc_d_s(9627824804082068411)[0] == 4813912402041034205
assert calc_d_s(9627824804082068411)[1] == 1
assert calc_d_s(203981900391787397947)[0] == 101990950195893698973
assert calc_d_s(203981900391787397947)[1] == 1



N, d, s = 221, 55, 2
#137 is a witness for compositeness of 221
a = 137
assert is_composite(N, d, s, a) == True

#174 will lead to 221 being a prime or 174 is a strong liar for 221
a = 174
assert is_composite(N, d, s, a) == False

N, d, s = 71, 35, 1
a = 19
assert is_composite(N, d, s, a) == False


assert is_prime_miller_rabin(203981900391787397947,40) == True

(p,q) is the private key with 512 bit primes, in question but books say (N,f) is private key

(N,e) is the public key, where N=pxq, 

h = hash(message)

computes signature
s = h^f mod(N)

f is the multiplicative inverse of e modulo φ(N)

send s to bob

In [19]:
def gen_modulas(n_bit):
    '''n_bit is the size of the primes you want, the secuirty level.
    This will output (N,p,q), N = p*q, p,q are n_bit uniform primes'''
    
    p=""
    q=""
    
    while type(p) == str or type(q) ==str:
        p = random_prime(n_bit, 40)
        q = random_prime(n_bit, 40)
    N = p*q
    return N, p, q

def choose_e(phi_n):
    e = 2
    while True:
        if gcd(e, phi_n) == 1:
            return e
        else:
            e += 1

def gen_RSA(n_bit):
    '''n_bit is the size of the primes you want, the secuirty level.
    output, N,e,f, N = p*q, e is relatively prime to phi(N), f solves ef = 1mod(phi(N))'''
    N, p, q = gen_modulas(n_bit)
    phi_n = (p-1)*(q-1)
    e = choose_e(phi_n)
    f = modular_inverse(e,phi_n)
    return N,e,f
    

In [20]:
assert choose_e(448) == 3

## The Plain RSA Signature Scheme
This is from the hash and sign paradigm. There are three stages to this scheme: Generation, Sign and Verify
+ Generation
  - We input the n-bit size of the random uniform primes we want
  - This will give us a private key $(N,f)$ and public key $(N,e)$
  - Where $N=pq$ and
  > $ef \equiv 1 \mod{\phi{N}} $
+ Sign
  - We input the private key $(N,f)$ and a messsage which will be hashed into $h$
  - So we can compute the $s \in \mathbb{Z}$, which is the signature
  > $s \equiv m^d \pmod{N}$  
+ Verify
  - We input the public key $(N,e)$ and a messsage which will be hashed into $h\prime$
  - We input $s$ the signature
  - Output $1$ which means authenticated if and only if
  > $h \equiv s^e \pmod{N}$

This works as
> $s^e \equiv (h^f)^e \equiv h^{ef \mod{\phi{N}}} \equiv h^1 \equiv h\mod{N}$

The reason why this is effective is becuase calculating $\phi{N}$ is very difficult, but with extra information $pq=N$ where $p,q \in$ Primes it makes $\phi{N}$ easy to calculate, this is know as a trap door. Additionally, the private key can be calculated from knowing the public key and the two primes that make up $N$.
    

In [23]:
def Generation(n_bit):
    '''input: n_bit, size of primes wanted
    output: private and public key'''
    N,e,f = gen_RSA(n_bit)
    private_key = {"N":N, "f":f}
    public_key = {"N":N, "e":e}
    return private_key, public_key
    
def Sign(message, private_key):
    h = my_hash(message)
    s = pow(h,private_key["f"],private_key["N"])
    return s

def Verify(message, s, public_key):
    h_prime = my_hash(message)
    h = pow(s, public_key["e"], public_key["N"])
    
    if h_prime == h:
        return 1 #Authenticated
    else:
        return 0 #Intruder
    

## Examples

### Example 1, no intruder

Bob wants to know the message he has been sent is from Alice. Therefore, Alice needs to generate private and public keys, sign the message. When Bob recieves the message he has to verify the message is from Alice.

In [29]:
message = "Hello Bob my freind"
#Generate keys with 512 bit primes
private_key, public_key = Generation(512)

#Alice needs to sign message
s = Sign(message, private_key)

#s and message are sent to Bob
#Bob should also have access to the public key
#Bob will now verify Alice sent the message
Verification = Verify(message, s, public_key)
Verification

1

As can be seen the verification process passed so Bob knows Alice sent the message.

Sometimes with very small probability the prime generator will produce a composite number. However, this is fine as then the message will fail the verification process and then the signature process happens again.

### Example 2, intruder Dave

Bob wants to know the message he has been sent is from Alice. And that Dave has not captured and changed the message. Same as before  Alice needs to generate private and public keys, sign the message. When Bob recieves the message he has to verify the message is from Alice. If the message fails verification Dave has changed the messsage. 

In [30]:
message = "Hello Bob my freind"
#Generate keys with 512 bit primes
private_key, public_key = Generation(512)

#Alice needs to sign message
s = Sign(message, private_key)

#s and message are sent to Bob
#Bob should also have access to the public key

#Dave intercepts the message and changes it
message = "I declare war on Bob"
#Dave sends new message to Bob

#Bob will now verify Alice sent the message
Verification = Verify(message, s, public_key)
Verification

0

The verification failed this means Bob knows that someone has intercepted and changed the message. 

Example 3, 10 bit primes no intruder
calc by hand show working
if test then we get all pass

Example 4, 10 bit primes no intruder Dave
calc by hand show working
if test then we get all pass


### Problems with RSA signature
Problem How do you know the public key is from Alice